Importamos las clases necesarias para extraer los datos relevabtes

In [25]:
import cv2
import mediapipe as mp
import csv
import itertools
import copy
import matplotlib.pyplot as plt
import os



In [26]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []
    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list


def logging_csv(number, mode, landmark_list,csv_path):
    if mode == 0:
        pass
    if mode == 1 and (0 <= number <= 9):
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *landmark_list])
    return


def extract_feature(dirt,number_gest):
    csv_path = 'model/keypoint_classifier/keypoint.csv'
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_hands = mp.solutions.hands
    with os.scandir(dirt) as ficheros:
        ficheros = [dirt+"\\"+fichero.name for fichero in ficheros if fichero.is_file()and fichero.name.endswith('.png')]
    # For static images:
    IMAGE_FILES = ficheros

    hands = mp_hands.Hands(
            static_image_mode='store_true',
            max_num_hands=1,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5,
        )
    for idx, file in enumerate(IMAGE_FILES):
        # Read an image, flip it around y-axis for correct handedness output (see
        # above).
        image = cv2.imread(file)    
        # Convert the BGR image to RGB before processing.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        if results.multi_hand_landmarks is not None:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                                    results.multi_handedness):
                landmark_list = calc_landmark_list(image, hand_landmarks)
                    # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(landmark_list)
        
                logging_csv(number_gest, 1, pre_processed_landmark_list,csv_path)
                # mp_drawing.draw_landmarks(
                #         image,
                #         hand_landmarks,
                #         mp_hands.HAND_CONNECTIONS,
                #         mp_drawing_styles.get_default_hand_landmarks_style(),
                #         mp_drawing_styles.get_default_hand_connections_style()
                #         )
                # plt.imshow(image.squeeze())
                # plt.axis("off")
                # plt.show()


In [27]:

class_group=[ "jump","right","left","fit","reset","back"]
dirt="dataset\\train\\"
label_path="model/keypoint_classifier/keypoint_classifier_label.csv"

for id_class,label in enumerate(class_group):
    extract_feature(dirt+label,id_class)
    with open(label_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([label])